In [1]:
import couchdb

In [2]:
couch = couchdb.Server('http://jb049/couchdb')
db = couch['livro_vermelho_2013']

### Exemplos de funcionamento

In [3]:
# Anneliesia cuneata (Lindl.) Senghas & LÃ¼ckel
doc1 = db.get('54ff1808dee85')
# Anneliesia candida (Lindl.) Brieger & Lückel
doc2 = db.get('54ff1808dedb9')
# Aulonemia aristulata (DÃ¶ll) McClure
doc3 = db.get('54ff180b4ff49')
# Aphelandra espirito-santensis Profice & Wassh.
doc4 = db.get('54bfbb681d248')
# Psychotria rhytidocarpa MÃ¼ll.Arg.
doc5 = db.get('54bfbb6fe1756')

In [4]:
print doc1['taxon']['scientificNameAuthorship']
print doc2['taxon']['scientificNameAuthorship']
print doc3['taxon']['scientificNameAuthorship']
print doc4['scientificNameAuthorship']
print doc5['scientificNameAuthorship']

(Lindl.) Senghas & LÃ¼ckel
(Lindl.) Brieger & Lückel
(DÃ¶ll) McClure
Profice & Wassh.
MÃÂ¼ll.Arg.


In [5]:
try:
    autor1 = doc1['taxon']['scientificNameAuthorship'].encode('iso-8859-1')
    print autor1.decode('utf-8')
except UnicodeDecodeError:
    print 'Autor 1 is ok.'
try:
    autor2 = doc2['taxon']['scientificNameAuthorship'].encode('iso-8859-1')
    print autor2.decode('utf-8')
except UnicodeDecodeError:
    print 'Autor 2 is ok.'
try:
    autor3 = doc3['taxon']['scientificNameAuthorship'].encode('iso-8859-1')
    print autor3.decode('utf-8')
except UnicodeDecodeError:
    print 'Autor 3 is ok.'
try:
    autor4 = doc4['scientificNameAuthorship'].encode('iso-8859-1')
    print autor4.decode('utf-8')
except UnicodeDecodeError:
    print 'Autor 4 is ok.'
try:
    autor5 = doc5['scientificNameAuthorship'].encode('iso-8859-1')
    print autor5.decode('utf-8')
except UnicodeDecodeError:
    print 'Autor 4 is ok.'

(Lindl.) Senghas & Lückel
Autor 2 is ok.
(Döll) McClure
Profice & Wassh.
MÃ¼ll.Arg.


In [29]:
doc1['taxon']['scientificNameAuthorship'] = autor1.decode('utf-8')
print 'Atualizando BD'
response = db.update([doc1])
print 'BD atualizado'
print response

Atualizando BD
BD atualizado
[(True, u'54ff1808dee85', u'6-d0dc1b2171b2a4e194f5a4894e1ebedb')]


### Script para recuperar e atualizar caracteres estranhos nos nomes do Livro Vermelho 2013

Campos: scientficName e scientificNameAuthorship

In [6]:
%%time
autor_ok = []
autor_fix = []
autor_fix_latin = []
autor_fix_only_latin = []
autor_fix_windows = []
autor_fix_windows_only = []
no_taxon = []
no_names = []
occurrences = []
attribute_error = []

updated = []
not_updated = []

i = 1

#doc_ids = ['54bfbb987bb24']

#for doc_id in doc_ids:
for doc_id in db:
    update = False
    doc = db.get(doc_id)
    
    if not (i % 5000):
        print 'Analisando registro', i
    
    if (doc['metadata']['type'] == 'profile') or (doc['metadata']['type'] == 'assessment'):
        try:
            # Autor is OK
            autor = doc['taxon']['scientificNameAuthorship'].decode('utf8')
            autor_ok.append(doc['_id'])
            update = False
        except UnicodeEncodeError:
            try:
                autor = doc['taxon']['scientificNameAuthorship'].encode('iso-8859-1')
                doc['taxon']['scientificNameAuthorship'] = autor.decode('utf-8')
                name = doc['taxon']['scientificName'].encode('iso-8859-1')
                doc['taxon']['scientificName'] = name.decode('utf-8')
                autor_fix.append(doc['_id'])
                update = True
                try:
                    autor = doc['taxon']['scientificNameAuthorship'].encode('windows_1252').decode(
                        'utf8').encode('latin1')
                    doc['taxon']['scientificNameAuthorship'] = autor.decode('utf-8')
                    name = doc['taxon']['scientificName'].encode('windows_1252').decode('utf8').encode('latin1')
                    doc['taxon']['scientificName'] = name.decode('utf-8')
                    autor_fix_windows.append(doc['_id'])
                except UnicodeEncodeError:
                    autor_fix_latin.append(doc['_id'])
                except UnicodeDecodeError:
                    try:
                        autor = doc['taxon']['scientificNameAuthorship'].encode('windows_1252')
                        doc['taxon']['scientificNameAuthorship'] = autor.decode('utf-8')
                        name = doc['taxon']['scientificName'].encode('windows_1252')
                        doc['taxon']['scientificName'] = name.decode('utf-8')
                        autor_fix_windows_only.append(doc['_id'])
                    except UnicodeDecodeError:
                        autor_fix_only_latin.append(doc['_id'])
            except UnicodeDecodeError:
                autor_ok.append(doc['_id'])
            except KeyError:
                no_taxon.append(doc['_id'])
            except AttributeError:
                attribute_error.append(doc['_id'])
        except AttributeError:
            attribute_error.append(doc['_id'])
    elif (doc['metadata']['type'] == 'taxon'):
        try:
            # Autor is OK
            autor = doc['scientificNameAuthorship'].decode('utf8')
            autor_ok.append(doc['_id'])
            update = False
        except UnicodeEncodeError:
            try:
                autor = doc['scientificNameAuthorship'].encode('iso-8859-1')
                doc['scientificNameAuthorship'] = autor.decode('utf-8')
                name = doc['scientificName'].encode('iso-8859-1')
                doc['scientificName'] = name.decode('utf-8')
                autor_fix.append(doc['_id'])
                update = True
                try:
                    autor = doc['scientificNameAuthorship'].encode('windows_1252').decode(
                        'utf8').encode('latin1')
                    doc['scientificNameAuthorship'] = autor.decode('utf-8')
                    name = doc['scientificName'].encode('windows_1252').decode('utf8').encode('latin1')
                    doc['scientificName'] = name.decode('utf-8')
                    autor_fix_windows.append(doc['_id'])
                except UnicodeEncodeError:
                    autor_fix_latin.append(doc['_id'])
                except UnicodeDecodeError:
                    try:
                        autor = doc['scientificNameAuthorship'].encode('windows_1252')
                        doc['scientificNameAuthorship'] = autor.decode('utf-8')
                        name = doc['scientificName'].encode('windows_1252')
                        doc['scientificName'] = name.decode('utf-8')
                        autor_fix_windows_only.append(doc['_id'])
                    except UnicodeDecodeError:
                        autor_fix_only_latin.append(doc['_id'])
            except UnicodeDecodeError:
                autor_ok.append(doc['_id'])
            except KeyError:
                no_names.append(doc['_id'])
            except AttributeError:
                attribute_error.append(doc['_id'])
        except AttributeError:
            attribute_error.append(doc['_id'])
    else:
        occurrences.append(doc['_id'])
        
    if update:
        db.update([doc])
        updated.append(doc['_id'])
    else:
        not_updated.append(doc['_id'])
    i+=1

Analisando registro 5000
Analisando registro 10000
Analisando registro 15000
Analisando registro 20000
Analisando registro 25000
Analisando registro 30000
Analisando registro 35000
Analisando registro 40000
Analisando registro 45000
Analisando registro 50000
Analisando registro 55000
Analisando registro 60000
Analisando registro 65000
Analisando registro 70000
Analisando registro 75000
Analisando registro 80000
Analisando registro 85000
Analisando registro 90000
Analisando registro 95000
Analisando registro 100000
Analisando registro 105000
Analisando registro 110000
Analisando registro 115000
Analisando registro 120000
Analisando registro 125000
Analisando registro 130000
Analisando registro 135000
Analisando registro 140000
Analisando registro 145000
Analisando registro 150000
Analisando registro 155000
Analisando registro 160000
Analisando registro 165000
Analisando registro 170000
Analisando registro 175000
Analisando registro 180000
Analisando registro 185000
Analisando registro 1

In [7]:
print "%d registros com autor OK" % len(autor_ok)
print "%d registros com autor consertado" % len(autor_fix)
print "%d registros com autor consertado em latin" % len(autor_fix_latin)
print "%d registros com autor consertado só latin" % len(autor_fix_only_latin)
print "%d registros com autor consertado em windows e latin" % len(autor_fix_windows)
print "%d registros com autor consertado só em windows" % len(autor_fix_windows_only)
print "%d registros dos tipos Assessment e Profile sem taxon" % len(no_taxon)
print "%d registros do tipo Taxon sem nomes" % len(no_names)
print "%d registros com AttributeError" % len(attribute_error)
print "%d registros do tipo Occurrence" % len(occurrences)
print "%d registros atualizados" % len(updated)
print "%d registros não atualizados" % len(not_updated)

24049 registros com autor OK
1026 registros com autor consertado
1 registros com autor consertado em latin
653 registros com autor consertado só latin
17 registros com autor consertado em windows e latin
355 registros com autor consertado só em windows
0 registros dos tipos Assessment e Profile sem taxon
0 registros do tipo Taxon sem nomes
519 registros com AttributeError
309482 registros do tipo Occurrence
1026 registros atualizados
334050 registros não atualizados


Campos: acceptedNameUsage

In [8]:
%%time
autor_ok_a = []
autor_fix_a = []
autor_fix_latin_a = []
autor_fix_only_latin_a = []
autor_fix_windows_a = []
autor_fix_windows_only_a = []
no_names_a = []
occurrences_a = []
attribute_error_a = []

updated_a = []
not_updated_a = []

i = 1

#doc_ids = ['54bfbb987bb24']

#for doc_id in doc_ids:
for doc_id in db:
    update = False
    doc = db.get(doc_id)
    
    if not (i % 5000):
        print 'Analisando registro', i
    
    if (doc['metadata']['type'] == 'taxon'):
        try:
            # Autor is OK
            autor = doc['acceptedNameUsage'].decode('utf8')
            autor_ok_a.append(doc['_id'])
            update = False
        except UnicodeEncodeError:
            try:
                autor = doc['acceptedNameUsage'].encode('iso-8859-1')
                doc['acceptedNameUsage'] = autor.decode('utf-8')
                autor_fix_a.append(doc['_id'])
                update = True
                try:
                    autor = doc['acceptedNameUsage'].encode('windows_1252').decode(
                        'utf8').encode('latin1')
                    doc['acceptedNameUsage'] = autor.decode('utf-8')
                    autor_fix_windows_a.append(doc['_id'])
                except UnicodeEncodeError:
                    autor_fix_latin_a.append(doc['_id'])
                except UnicodeDecodeError:
                    try:
                        autor = doc['acceptedNameUsage'].encode('windows_1252')
                        doc['acceptedNameUsage'] = autor.decode('utf-8')
                        autor_fix_windows_only_a.append(doc['_id'])
                    except UnicodeDecodeError:
                        autor_fix_only_latin_a.append(doc['_id'])
            except UnicodeDecodeError:
                autor_ok_a.append(doc['_id'])
            except KeyError:
                no_names_a.append(doc['_id'])
            except AttributeError:
                attribute_error_a.append(doc['_id'])
        except AttributeError:
            attribute_error_a.append(doc['_id'])
    else:
        occurrences_a.append(doc['_id'])
        
    if update:
        db.update([doc])
        updated_a.append(doc['_id'])
    else:
        not_updated_a.append(doc['_id'])
    i+=1

Analisando registro 5000
Analisando registro 10000
Analisando registro 15000
Analisando registro 20000
Analisando registro 25000
Analisando registro 30000
Analisando registro 35000
Analisando registro 40000
Analisando registro 45000
Analisando registro 50000
Analisando registro 55000
Analisando registro 60000
Analisando registro 65000
Analisando registro 70000
Analisando registro 75000
Analisando registro 80000
Analisando registro 85000
Analisando registro 90000
Analisando registro 95000
Analisando registro 100000
Analisando registro 105000
Analisando registro 110000
Analisando registro 115000
Analisando registro 120000
Analisando registro 125000
Analisando registro 130000
Analisando registro 135000
Analisando registro 140000
Analisando registro 145000
Analisando registro 150000
Analisando registro 155000
Analisando registro 160000
Analisando registro 165000
Analisando registro 170000
Analisando registro 175000
Analisando registro 180000
Analisando registro 185000
Analisando registro 1

In [9]:
print "%d registros com autor OK" % len(autor_ok_a)
print "%d registros com autor consertado" % len(autor_fix_a)
print "%d registros com autor consertado em latin" % len(autor_fix_latin_a)
print "%d registros com autor consertado só latin" % len(autor_fix_only_latin_a)
print "%d registros com autor consertado em windows e latin" % len(autor_fix_windows_a)
print "%d registros com autor consertado só em windows" % len(autor_fix_windows_only_a)
print "%d registros do tipo Taxon sem nomes" % len(no_names_a)
print "%d registros com AttributeError" % len(attribute_error_a)
print "%d registros dos tipos Occurrence, Assessment ou Profile" % len(occurrences_a)
print "%d registros atualizados" % len(updated_a)
print "%d registros não atualizados" % len(not_updated_a)

15177 registros com autor OK
533 registros com autor consertado
0 registros com autor consertado em latin
18 registros com autor consertado só latin
0 registros com autor consertado em windows e latin
515 registros com autor consertado só em windows
0 registros do tipo Taxon sem nomes
4 registros com AttributeError
319362 registros dos tipos Occurrence, Assessment ou Profile
533 registros atualizados
334543 registros não atualizados


In [111]:
print 5263+4617+309482
print 15177+533+4
print 515+18

319362
15714
533


### Exemplos que não deram certo (feedback para melhorar o algoritmo)

In [8]:
taxon = []
profile = []
assessment = []
for doc_id in autor_fix:
    doc = db.get(doc_id)
    if (doc['metadata']['type'] == 'profile'):
        profile.append(doc['_id'])
    elif (doc['metadata']['type'] == 'assessment'):
        assessment.append(doc['_id'])
    elif (doc['metadata']['type'] == 'taxon'):
        taxon.append(doc['_id'])
    else:
        print "Error %d", doc['_id']

In [ ]:
print "%d taxons" % len(taxon)
print "%d assessements" % len(assessment)
print "%d profiles" % len(profile)

In [27]:
%%time
accepted = {"encode": [], "decode": []}
scientific = {"encode": [], "decode": []}
author = {"encode": [], "decode": []}
for taxon_id in taxon:
    doc = db.get(taxon_id)
    try:
        autor = doc['acceptedNameUsage'].encode('iso-8859-1')
        autor.decode('utf-8')
    except UnicodeDecodeError:
        accepted['decode'].append(taxon_id)
    except UnicodeEncodeError:
        accepted['encode'].append(taxon_id)
    try:
        autor = doc['scientificName'].encode('iso-8859-1')
        autor.decode('utf-8')
    except UnicodeDecodeError:
        scientific['decode'].append(taxon_id)
    except UnicodeEncodeError:
        scientific['encode'].append(taxon_id)
    try:
        autor = doc['scientificNameAuthorship'].encode('iso-8859-1')
        autor.decode('utf-8')
    except UnicodeDecodeError:
        author['decode'].append(taxon_id)
    except UnicodeEncodeError:
        author['encode'].append(taxon_id)

#### Consertado em accepted mas não em scientific

In [22]:
set(accepted['decode']) - set(scientific['decode'])

{u'54bfbb7b1353e',
 u'54bfbb7b135a8',
 u'54bfbb7b135f5',
 u'54bfbb7b13641',
 u'54bfbb7b1368c',
 u'54bfbb7f3e862',
 u'54bfbb7f925cb',
 u'54bfbb85b081c'}

####Consertado em scientific mas não em accepted

In [28]:
set(scientific['decode']) - set(accepted['decode'])

{u'54bfbb7214176',
 u'54bfbb7214ac2',
 u'54bfbb771fb0d',
 u'54bfbb771fb68',
 u'54bfbb77203f7',
 u'54bfbb772056f',
 u'54bfbb772a9b7',
 u'54bfbb7738315',
 u'54bfbb7788500',
 u'54bfbb778854e',
 u'54bfbb77ab260',
 u'54bfbb77aba12',
 u'54bfbb77abc1e',
 u'54bfbb77abc69',
 u'54bfbb77b971f',
 u'54bfbb77b976c',
 u'54bfbb77d5f4a',
 u'54bfbb77e3354',
 u'54bfbb77e3cf4',
 u'54bfbb7839a67',
 u'54bfbb78e9130',
 u'54bfbb790393c',
 u'54bfbb790f9a6',
 u'54bfbb790fad5',
 u'54bfbb790fb21',
 u'54bfbb790fbb7',
 u'54bfbb791fa67',
 u'54bfbb791fb20',
 u'54bfbb791ffc7',
 u'54bfbb7934c7f',
 u'54bfbb7934e7d',
 u'54bfbb7934f13',
 u'54bfbb793516d',
 u'54bfbb7942222',
 u'54bfbb7942a1f',
 u'54bfbb7942a6b',
 u'54bfbb7942ab6',
 u'54bfbb7942b4d',
 u'54bfbb795af11',
 u'54bfbb7966f41',
 u'54bfbb7966f90',
 u'54bfbb7967074',
 u'54bfbb79670bf',
 u'54bfbb796710a',
 u'54bfbb7967154',
 u'54bfbb79672c9',
 u'54bfbb79673f3',
 u'54bfbb79674d2',
 u'54bfbb796751c',
 u'54bfbb79675b0',
 u'54bfbb7967666',
 u'54bfbb79676b1',
 u'54bfbb796

In [29]:
scientific['encode']

[u'54bfbb7fc0cbd',
 u'54bfbb7fda8b1',
 u'54bfbb7fed1e3',
 u'54bfbb801efc7',
 u'54bfbb802de78',
 u'54bfbb8588e89',
 u'54bfbb8588ed7',
 u'54bfbb8588f25',
 u'54bfbb8588f73',
 u'54bfbb8595de8',
 u'54bfbb85e1cdb',
 u'54bfbb85eccfa',
 u'54bfbb85ecd95',
 u'54bfbb85ecf8b',
 u'54bfbb85ed090',
 u'54bfbb861f7f2',
 u'54bfbb861fc28',
 u'54bfbb987bb24']

In [126]:
# -*- coding: utf-8 -*-
doc = db.get('54bfbb7fc0cbd')
print doc['scientificNameAuthorship']
autor = doc['scientificNameAuthorship'].encode('iso-8859-5')
#print autor.encode('windows-1252')
print autor
#print chr(192)
#print ord('')
    #print autor.decode('utf-8')
    #doc['scientificNameAuthorship'] = autor.decode('utf-8')
    #name = doc['scientificName'].encode('iso-8859-1')
    #print name.decode('utf-8')
    #doc['scientificName'] = name.decode('utf-8')
    #accepted_autor = doc['acceptedNameUsage'].encode('iso-8859-1')
    #doc['acceptedNameUsage'] = accepted_autor.decode('utf-8')
    #print accepted.decode('utf-8')
#except UnicodeDecodeError:
 #   print 'Autor 1 is ok.'

(RuÃƒÂ­z & Pav.) Chodat


UnicodeEncodeError: 'charmap' codec can't encode characters in position 3-5: character maps to <undefined>

In [109]:
from encodings.aliases import aliases

In [148]:
final_results = []
for enc in aliases.items():
    try:
        autor= doc['scientificNameAuthorship'].encode(enc[0]).decode('latin1')
        print enc[0]
        print autor
        final_results.append(autor)
    except UnicodeEncodeError:
        pass
        #print "%s cant encode" % enc[1]
    except LookupError:
        pass
        #print "cant find encode %s" % enc[1]
    except UnicodeDecodeError:
        pass
        #print "cant decode %s" % enc[1]

1140
MÙ¤f#f"bÊ©@P@×¥K]@Ã£
iso8859
(RuÃÆÃÂ­z & Pav.) Chodat
iso_celtic
(RuÃÆÃÂ­z & Pav.) Chodat
utf32
ÿþ  (   R   u   Ã      Æ      Ã      Â   ­   z       &       P   a   v   .   )       C   h   o   d   a   t   
ebcdic_cp_wt
MÙ¤f#f"bÊ©@P@×¥K]@Ã£
ibm1140
MÙ¤f#f"bÊ©@P@×¥K]@Ã£
csibm1026
MÙ¤f#f"bÊ©@P@×¥K]@Ã£
rot13
(EhÃÆÃÂ­m & Cni.) Pubqng
ibm1026
MÙ¤f#f"bÊ©@P@×¥K]@Ã£
utf_16le
( R u Ã  Æ  Ã  Â ­ z   &   P a v . )   C h o d a t 
csibm500
MÙ¤f#f"bÊ©@P@×¥K]@Ã£
u32
ÿþ  (   R   u   Ã      Æ      Ã      Â   ­   z       &       P   a   v   .   )       C   h   o   d   a   t   
l5
(RuÃÆÃÂ­z & Pav.) Chodat
ibm500
MÙ¤f#f"bÊ©@P@×¥K]@Ã£
l8
(RuÃÆÃÂ­z & Pav.) Chodat
utf_16be
 ( R u Ã  Æ  Ã  Â ­ z   &   P a v . )   C h o d a t
latin
(RuÃÆÃÂ­z & Pav.) Chodat
gb18030_2000
(Ru0103040801020002z & Pav.) Chodat
ebcdic_cp_us
MÙ¤f#f"bÊ©@P@×¥K]@Ã£
ebcdic_cp_nl
MÙ¤f#f"bÊ©@P@×¥K]@Ã£
csisolatin1
(RuÃÆÃÂ­z & P

In [143]:
for doc_id in scientific['encode']:
    try:
        doc = db.get(doc_id)
        autor= doc['scientificNameAuthorship'].encode('windows_1254').decode('utf-8').encode('latin1')
        print autor
    except UnicodeEncodeError:
        print doc['scientificNameAuthorship']

UnicodeEncodeError: 'charmap' codec can't encode character u'\x83' in position 4: character maps to <undefined>

In [120]:
print final_results

[u'K\xc3\xb6rn.', u'K\xc3\xb6rn.', u'K+AMMBkgDCALY-rn.', u'K\xc3\u0192\xc2\xb6rn.', u'K+AMMBkgDCALY-rn.', u'K\xc3\xb6rn.', u'K\xc3\xb6rn.', u'K\xc3\u0192\xc2\xb6rn.', u'K\xc3\u0192\xc2\xb6rn.', u'K\xc3\u0192\xc2\xb6rn.', u'K\xc3\u0192\xc2\xb6rn.', u'K+AMMBkgDCALY-rn.']


In [110]:
aliases.items()

[('iso_ir_6', 'ascii'),
 ('1140', 'cp1140'),
 ('tis620', 'tis_620'),
 ('iso8859', 'latin_1'),
 ('chinese', 'gb2312'),
 ('mskanji', 'cp932'),
 ('s_jis', 'shift_jis'),
 ('iso_celtic', 'iso8859_14'),
 ('utf32', 'utf_32'),
 ('ebcdic_cp_wt', 'cp037'),
 ('csibm863', 'cp863'),
 ('ebcdic_cp_he', 'cp424'),
 ('csHPRoman8', 'hp_roman8'),
 ('cp936', 'gbk'),
 ('thai', 'iso8859_11'),
 ('maccyrillic', 'mac_cyrillic'),
 ('csibm857', 'cp857'),
 ('iso_8859_6_1987', 'iso8859_6'),
 ('csibm855', 'cp855'),
 ('ibm1140', 'cp1140'),
 ('eucjis2004', 'euc_jis_2004'),
 ('csibm1026', 'cp1026'),
 ('ansi_x3_4_1968', 'ascii'),
 ('dbcs', 'mbcs'),
 ('rot13', 'rot_13'),
 ('ibm367', 'ascii'),
 ('csibm858', 'cp858'),
 ('eucjp', 'euc_jp'),
 ('latin10', 'iso8859_16'),
 ('shiftjisx0213', 'shift_jisx0213'),
 ('shiftjis2004', 'shift_jis_2004'),
 ('932', 'cp932'),
 ('ibm1026', 'cp1026'),
 ('utf_16le', 'utf_16_le'),
 ('csibm500', 'cp500'),
 ('u32', 'utf_32'),
 ('iso_ir_127', 'iso8859_6'),
 ('uhc', 'cp949'),
 ('iso_8859_11', 'iso

In [11]:
'54bfbb865e4ab' in taxon

True

In [12]:
'54ff1808dee85' in profile

False

In [63]:
doc_id = '54bfbb7fc0cbd'
doc_id =  '54ff1808dee85'
#doc_id = '54ff17fd53601'
doc_id = '54bfbb6fe1756'
doc = db.get(doc_id)
#print doc['taxon']['scientificNameAuthorship']
print doc['scientificNameAuthorship']
#autor= doc['scientificNameAuthorship'].encode('windows_1254').decode('utf-8').encode('latin1')
#autor= doc['scientificNameAuthorship'].decode('utf8')
#print autor
#autor= doc['taxon']['scientificNameAuthorship'].encode('latin1').decode('utf8')
autor= doc['scientificNameAuthorship'].encode('latin1').decode('utf8')
autor = autor.encode('windows_1254').decode('utf-8')
print autor
#.encode('latin-1').decode('utf8')

#.decode('utf-8').encode('latin1').decode("utf8")
#print autor

MÃÂ¼ll.Arg.
Müll.Arg.


In [1]:
import pickle

In [38]:
pickle.dump( autor_fix, open( "author_fixed.pkl", "wb" ) )

In [4]:
autor_fix = pickle.load(open( "author_fixed.pkl", "rb" ))

#### Para testar
54bfbb7b13641